In [40]:
import pandas as pd
import json
import urllib
import os
import numpy as np

In [8]:
data = pd.read_excel('/Users/toni_brain/Documents/Uni/Collaborations/Christine_Mayr/RBH_final_221222.xlsx')

In [13]:
ids = data['uniprot_id'].tolist()

In [14]:
unique_ids = list(set(ids))

In [16]:
len(unique_ids)

16238

In [45]:
f = open('unique_ids_from_spreadsheet.txt','w')
for i in unique_ids:
    f.write(f'{i}\n')
f.close()

In [46]:
def download_uniprot_json_file(uni_prot_id, workdir = '.'):
    #check if there is uniprot information available for the protein
    try:
        url_2 = 'https://www.uniprot.org/uniprot/' + uni_prot_id + '.json'
        html_2 = urllib.request.urlopen(url_2)
        lines = html_2.readlines()[0]
        # now try and write to file
        f = open(os.path.join(workdir,uni_prot_id)+'.json', 'w')
        f.write(lines.decode('utf-8'))
        f.close()
        
    except Exception as e:
        raise Exception('Failed to obtain UNIPROT data. %s'%e)
    

    return html_2

In [70]:
def get_kown_things(known_things,data):

    #known_things['Helix'] = []
    if 'features' in data.keys():
        for d in data['features']:
            #print(d['type'])
            if d['type'] == 'Domain':
                if d['description'] not in  known_things['Domain']:
                    known_things['Domain'].append(d['description'])
            elif d['type'] == 'Region':
                if d['description'] not in  known_things['Region']:
                    known_things['Region'].append( d['description'])
            elif d['type'] == 'Motif':
                if d['description'] not in  known_things['Motif']:
                    known_things['Motif'].append(d['description'])
    return known_things

In [ ]:
known_things = {}
known_things['Domain'] = []
known_things['Region'] = []
known_things['Motif'] = []

In [71]:
known_things = info

In [72]:

counter = 11857
for ids in unique_ids[11858:]:
    download_uniprot_json_file(ids, workdir='unique_ids')
    # Process uniprot file
    f = open('unique_ids/'+ids+'.json')
    data = json.load(f)
    f.close()
    info = get_kown_things(known_things,data)
    counter = counter +1
    if counter%500==0:
        print(f'At entry {counter}/{len(unique_ids)}')

At entry 12000/16238
At entry 12500/16238
At entry 13000/16238
At entry 13500/16238
At entry 14000/16238
At entry 14500/16238
At entry 15000/16238
At entry 15500/16238
At entry 16000/16238


In [76]:
info_keys = info.keys()

In [100]:
info

{'Domain': ['SH3',
  'BRO1',
  'SOCS box',
  'F-box',
  'Ricin B-type lectin',
  'BTB',
  'BEN',
  'EF-hand 1',
  'EF-hand 2',
  'EF-hand 3',
  'Ferric oxidoreductase',
  'FAD-binding FR-type',
  'Laminin G-like 1',
  'EGF-like 1',
  'Laminin G-like 2',
  'Laminin G-like 3',
  'EGF-like 2',
  'Laminin G-like 4',
  'Laminin G-like 5',
  'EGF-like 3',
  'Laminin G-like 6',
  'Protein kinase',
  'AGC-kinase C-terminal',
  'PDZ',
  'SUEL-type lectin',
  'Olfactomedin-like',
  'GPS',
  'Ig-like C1-type',
  'GST C-terminal',
  'PH',
  'RUN',
  'Thioredoxin',
  'VWFA 1',
  'EGF-like 1; incomplete',
  'EGF-like 4',
  'VWFA 2',
  'EGF-like 5; calcium-binding',
  'EGF-like 6',
  'U-box',
  'Ig-like',
  'LRRNT',
  'C2 1',
  'C2 2',
  'ADF-H 1',
  'ADF-H 2',
  'Ig-like C2-type',
  'SAM',
  'Fibronectin type-III 1',
  'Fibronectin type-III 2',
  'Tyrosine-protein phosphatase',
  'EF-hand 4',
  'EF-hand 5',
  'EF-hand 6',
  'Helicase ATP-binding',
  'Helicase C-terminal',
  'RRM',
  'UIM',
  'UBX',


In [77]:
for k in info_keys:
    print(set(info[k]))

{'Ig-like C2-type', 'TSP type-1 17', 'tRNA-binding', 'PAH 3', 'PSI', 'AXH', 'P-type', 'MDFI', 'HTH OST-type 6', 'NHR 1', 'IQ 5', 'MI 2', 'EGF-like 6', 'IQ', 'SCAN box', 'EGF-like 8; calcium-binding', "Beta/gamma crystallin 'Greek key' 12", 'S1 motif 11', 'TSP type-1 13', 'DAZ 1', 'TsaA-like', 'EGF-like 2; calcium-binding', 'MAGE', 'LDL-receptor class A 5', 'Olduvai 24', 'KRAB 2', 'NtA', 't-SNARE coiled-coil homology', "3'-5' exonuclease", 'FF 2', 'Prospero', 'Myosin N-terminal SH3-like', 'Cadherin 11', 'Zinc-hook', 'Fe2OG dioxygenase', 'Cytochrome b561', 'Tudor-knot', 'RhoBD', 'Teneurin N-terminal', 'Calx-beta 1', 'Pentapeptide repeat 1', 'SEA', 'WAP 2', 'BEN 3', 'UBA', 'SF4 helicase', 'EGF-like 17; calcium-binding', 'WAP 4', 'JmjN', 'Fibrillar collagen NC1', 'v-SNARE coiled-coil homology', 'PKD 3', 'ASD1', 'HTH cro/C1-type', 'BSD 1', 'KOW 5', 'Peptidase S1 3', 'DOMON', 'Laminin EGF-like 11', 'BRCT 3', 'SLD', 'DMA', 'Cache 1', 'ADF-H', 'Histone-fold; involved in forming hexamer structu

In [89]:
domain = list(set(info['Domain']))

In [102]:
for d in domain:
    if 'PUM-HD'.lower() in d.lower():
        print('Pumilio')
    if 'HEAT'.lower() in d.lower():
        print('HEAT')
    if 'ARM'.lower() in d.lower():
        print('ARM') 
    if 'bHLH'.lower() in d.lower():
        print('bHLH')

bHLH
BIG
Pumilio


In [98]:
region = list(set(info['Region']))

In [99]:
for d in region:
    if 'Pumilio' in d:
        print('Pumilio')
    if 'HEAT' in d:
        print('HEAT')
    if 'ARM' in d:
        print('ARM') 
    if 'bHLH' in d:
        print('bHLH')
    if 'BIG' in d:
        print('BIG')

ARM
HEAT
ARM
ARM
ARM


In [93]:
print('Pumilio' in region)
print('HEAT' in region)
print('ARM' in region) 
print('bHLH' in region)
print('EXACT' in region)

False
False
False
False
False


In [103]:
len(region)

4514

In [104]:
len(domain)

1613

In [106]:
motif = list(set(info['Motif']))

In [108]:
len(motif)

718

In [111]:
np.savetxt('motif.csv', motif, fmt='%s')

In [112]:
np.savetxt('region.csv', region, fmt='%s')

In [114]:
np.savetxt('domain.csv', domain, fmt='%s')

In [81]:
np.save('info.npy', info)

In [68]:
unique_ids.index('Q8WV35')

11857

In [69]:
unique_ids[11857]

'Q8WV35'